In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import sklearn.metrics as metric
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBClassifier
import json
import joblib
import os

In [2]:
fpkm = pd.read_csv("../datasets/datasets_processed/fpkm/toden_fpkm.txt", sep="\t", index_col=0)
tpm = pd.read_csv("../datasets/datasets_processed/tpm/toden_tpm.txt", sep="\t", index_col=0)
raw_count = pd.read_csv("../datasets/datasets_raw/toden/toden_counts.txt", sep="\t")
meta = pd.read_excel("../datasets/datasets_raw/toden/toden_metadata.xlsx")

In [3]:
# Convert categorical variable ['Apoe.status','apoe_carrier','apoe_dose']
share_feat = ['Apoe.status','apoe_carrier','apoe_dose']
status_dic = {'None': 0, 'E2/E3': 1, 'E2/E4': 2, 'E3/E3': 3, 'E3/E4': 4, 'E4/E4': 5}
carrier_dic = {'None': 0, 'no_apoe4': 0, 'apoe4': 1}
dose_dic = {'apoe4': 1, 'None': 0, 'apoe44': 2, 'no_apoe4': 0}
com_dic = [status_dic, carrier_dic, dose_dic]

meta_feat = meta[share_feat].copy()
for i in range(len(share_feat)):
    meta_feat[share_feat[i]] = meta_feat[share_feat[i]].map(com_dic[i])
meta_feat.fillna(0, inplace=True)

In [4]:
tab1 = pd.read_excel("../datasets/abb1654_data_file_s1.xlsx", sheet_name="Upregulated genes", header=1)
tab2 = pd.read_excel("../datasets/abb1654_data_file_s1.xlsx", sheet_name="Downregulated genes", header=1)
tab3 = pd.read_csv('/Users/lisirui/Desktop/BDA/project/BENG-203-Project/datasets/hvg_genes.txt', header=None)
select_gene = list(set(list(tab1['ID'])+list(tab2['ID'])))
hvg_genes = tab3[0].tolist()

In [5]:
# Prepare training data (x)
df1 = raw_count.T
df2 = df1.copy()
df3 = df1.copy()[list(set(df1.columns) & set(select_gene))]
df4 = df3.copy()
df5 = df1.copy()[list(set(df1.columns) & set(hvg_genes))]
df6 = df5.copy()

df7 = fpkm.copy()
df8 = fpkm.copy()
df9 = fpkm.copy()[list(set(df7.columns) & set(select_gene))]
df10 = df9.copy()
df11 = fpkm.copy()[list(set(df7.columns) & set(hvg_genes))]
df12 = df11.copy()

df13 = tpm.copy()
df14 = tpm.copy()
df15 = tpm.copy()[list(set(df13.columns) & set(select_gene))]
df16 = df15.copy()
df17 = tpm.copy()[list(set(df13.columns) & set(hvg_genes))]
df18 = df17.copy()

for i in meta_feat.columns:
    df2[i] = meta_feat[i].values
    df4[i] = meta_feat[i].values
    df6[i] = meta_feat[i].values
    df8[i] = meta_feat[i].values
    df10[i] = meta_feat[i].values
    df12[i] = meta_feat[i].values
    df14[i] = meta_feat[i].values
    df16[i] = meta_feat[i].values
    df18[i] = meta_feat[i].values

train_input=['count', 'count+apoe', 'select-gene count', 'select-gene count+apoe', 'hvg-gene count', 'hvg-gene count+apoe',
             'FPKM', 'FPKM+apoe', 'select-gene FPKM', 'select-gene FPKM+apoe', 'hvg-gene FPKM', 'hvg-gene FPKM+apoe',
             'TPM', 'TPM+apoe', 'select-gene TPM', 'select-gene TPM+apoe', 'hvg-gene TPM', 'hvg-gene TPM+apoe']
train_data = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18]

In [6]:
import warnings
warnings.filterwarnings('ignore')

# extract y
y = meta['Disease']
ma = {'NCI': 0, 'AD': 1}
for i in range(len(y)):
    y.iloc[i] = ma[y.iloc[i]]
y = y.astype(int)

In [7]:
norm_data = {}
for i in ["l1", "l2"]:
    norm_data[i] = []
    for j in range(6):
        if j % 2 == 0:
            norm_data[i].append(pd.DataFrame(normalize(train_data[j], norm=i, axis=1), columns=train_data[j].columns, index=train_data[j].index))
        else:
            temp = normalize(train_data[j].iloc[:, :-3], norm=i, axis=1)
            temp_df = pd.DataFrame(temp, columns=train_data[j].columns[:-3], index=train_data[j].index)
            norm_data[i].append(pd.concat([temp_df, train_data[j].iloc[:, -3:]], axis=1))

In [8]:
def save_model(model, model_name):
    os.makedirs("../regression_models/", exist_ok=True)
    joblib.dump(model, os.path.join("../regression_models/", f"{model_name}.pkl"))

def load_model(model_name):
    return joblib.load(os.path.join("../regression_models/", f"{model_name}.pkl"))

In [9]:
def regression_models(train_data, train_input, y, model_type='linear', norm=None, results=None):
    for i in range(len(train_data) if norm == None else 6):
        dTrain = train_data[i]

        if norm != None:
            dTrain = norm_data[norm][i]
        
        for k, (trainInd, valInd) in enumerate(KFold(shuffle=True, random_state=16).split(dTrain)):
            XTrain = dTrain.iloc[trainInd,]
            yTrain = y.iloc[trainInd,]
            XVal = dTrain.iloc[valInd,]
            yVal = y.iloc[valInd,]

            if model_type == 'linear':
                model = LinearRegression()
            else:
                model = LogisticRegression(max_iter=1000)
                
            model.fit(XTrain, yTrain)
            pred = model.predict(XVal)
            pred_binary = (pred >= 0.5).astype(int)
            
            result_key = f"{model_type}_regression_{norm if norm else 'no'}_norm_{train_input[i]}_{k}"
            save_model(model, result_key)
            
            if model_type == 'linear':
                results[result_key] = {
                    "accuracy": metric.r2_score(yVal, pred_binary),
                }
                feature_importance_df = pd.DataFrame({
                    'Feature': dTrain.columns,
                    'Importance': model.coef_
                }).sort_values(by='Importance', ascending=False)
                results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')
            else:
                results[result_key] = {
                    "accuracy": metric.accuracy_score(yVal, pred_binary),
                    "roc_auc": metric.roc_auc_score(yVal, model.predict_proba(XVal)[:,1]),
                    "f1_score": metric.f1_score(yVal, pred_binary),
                    "mcc_score": metric.matthews_corrcoef(yVal, pred_binary),
                    "recall": metric.recall_score(yVal, pred_binary)
                }
                feature_importance_df = pd.DataFrame({
                    'Feature': dTrain.columns,
                    'Importance': model.coef_[0]
                }).sort_values(by='Importance', ascending=False)
                results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')

In [10]:
results = {}
for i in ['linear', 'logistic']:
    regression_models(train_data, train_input, y, model_type=i, results=results)
    for j in ['l1', 'l2']:
        regression_models(train_data, train_input, y, model_type=i, norm=j, results=results)

In [11]:
def xgboost_model(train_data, train_input, y, norm=None, results=None):
    for i in range(len(train_data) if norm is None else 6):
        dTrain = train_data[i]

        if norm is not None:
            dTrain = norm_data[norm][i]
        
        for k, (trainInd, valInd) in enumerate(KFold(shuffle=True, random_state=16).split(dTrain)):
            XTrain = dTrain.iloc[trainInd]
            yTrain = y.iloc[trainInd]
            XVal = dTrain.iloc[valInd]
            yVal = y.iloc[valInd]

            model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
            model.fit(XTrain, yTrain)
            pred = model.predict(XVal)
            pred_binary = (pred >= 0.5).astype(int)

            result_key = f"xgboost_{norm if norm else 'no'}_norm_{train_input[i]}_{k}"
            save_model(model, result_key)

            results[result_key] = {
                "accuracy": metric.accuracy_score(yVal, pred_binary),
                "roc_auc": metric.roc_auc_score(yVal, model.predict_proba(XVal)[:,1]),
                "f1_score": metric.f1_score(yVal, pred_binary),
                "mcc_score": metric.matthews_corrcoef(yVal, pred_binary),
                "recall": metric.recall_score(yVal, pred_binary)
            }
            feature_importance_df = pd.DataFrame({
                'Feature': dTrain.columns,
                'Importance': model.feature_importances_
            }).sort_values(by='Importance', ascending=False)
            results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')

In [12]:
xgboost_model(train_data, train_input, y, results=results)
for i in ['l1', 'l2']:
    xgboost_model(train_data, train_input, y, norm=i, results=results)

In [13]:
with open('regression_results.json', 'w') as f:
    json.dump(results, f, indent=4)

In [14]:
for key, value in results.items():
    print(f"\nResults for {key}:\n")
    for k, v in value.items():
        if k != "feature_importance":
            print(f"{k}: {v}")
    print("Top 5 Important Features:\n", pd.DataFrame(value["feature_importance"][:5]))


Results for linear_regression_no_norm_count_0:

accuracy: 0.34196428571428583
Top 5 Important Features:
            Feature  Importance
0  ENSG00000170315    0.000052
1  ENSG00000229754    0.000052
2  ENSG00000198899    0.000046
3  ENSG00000198763    0.000044
4  ENSG00000206503    0.000043

Results for linear_regression_no_norm_count_1:

accuracy: 0.3920145190562613
Top 5 Important Features:
            Feature  Importance
0  ENSG00000198899    0.000057
1  ENSG00000086232    0.000049
2  ENSG00000181222    0.000049
3  ENSG00000184678    0.000047
4  ENSG00000170315    0.000045

Results for linear_regression_no_norm_count_2:

accuracy: -0.1787037037037038
Top 5 Important Features:
            Feature  Importance
0  ENSG00000170315    0.000069
1  ENSG00000198786    0.000061
2  ENSG00000213741    0.000048
3  ENSG00000168461    0.000044
4  ENSG00000184678    0.000042

Results for linear_regression_no_norm_count_3:

accuracy: -0.23352713178294593
Top 5 Important Features:
            Feature